<a href="https://colab.research.google.com/github/R3dh0od/simgrid/blob/main/transformer_psentymento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
root='/content/drive/My Drive/Colab Notebooks/NLP/EXIST'

Mounted at /content/drive/


In [ ]:
import pandas as pd
import os
import numpy as np
import re
import pickle

LIMPIEZA DE DATOS

In [ ]:
def get_data(path):
    save_path = path + '.dict'
    if os.path.exists(save_path):
        return pickle.load(open(save_path, 'rb'))
    else:
        df = pd.read_csv(os.path.join(root, path), sep='\t')

        df['task1'] = df['task1'].map(task1)
        df['task2'] = df['task2'].map(task2)
        df['text'] = df['text'].map(lambda t: re.sub('@[\w\d]+', '', t)) # remove user names
        df['text'] = df['text'].map(lambda t: re.sub('https://t\.co/[\w\d]+', '', t)) # remove links
        df['text'] = df['text'].map(lambda t: re.sub('&amp;', 'and', t)) # change & twitter symbol for and
        df['text'] = df['text'].map(lambda t: re.sub('&lt;', '', t)) # change < twitter symbol for lower than
        df['text'] = df['text'].map(lambda t: re.sub('&gt;', '', t)) # change > twitter symbol for greater than
        lang = {}
        for l, df_lang in df.groupby('language'):
            df_lang['text_vec'] = df_lang['text'].map(lambda t: nlp[l](t).vector)
            df_lang['word_vec'] = df_lang['text'].map(lambda t: np.asarray([token.vector for token in nlp[l](t)]))
            lang[l] = df_lang.drop(columns=['test_case', 'id', 'source', 'language'])
        pickle.dump(lang, open(save_path, 'wb'))
        return lang

In [ ]:
train = get_data(root+'/data/EXIST2021_training.tsv')['en']
test = get_data(root+'/data/EXIST2021_test.tsv')['en']

In [ ]:
X1=list(test['text'])
X2=list(train['text'])
X=X1+X2
y1=list(test['task1'])
y2=list(train['task1'])
y=y1+y2

## Splitting the dataset

In [ ]:
# Splitting into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# Install hugginface transformers library

let's install Huggingface transformers library along with others:


In [ ]:
!pip3 install transformers numpy torch sklearn


     |████████████████████████████████| 4.0 MB 3.9 MB/s 
     |████████████████████████████████| 895 kB 17.9 MB/s 
     |████████████████████████████████| 596 kB 34.9 MB/s 
     |████████████████████████████████| 6.6 MB 33.1 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Import the necessary modules:

In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis").to("cuda")

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Now let's use our tokenizer to encode our corpus. We set truncation to True so that we eliminate tokens that go above max_length, we also set padding to True to pad documents that are less than max_length with empty tokens.

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
max_length=512
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=max_length)

The below code wraps our tokenized text data into a torch Dataset. Since we gonna use Trainer from Transformers library, it expects our dataset as a torch.utils.data.Dataset, so we made a simple class that implements the __len__() method that returns the number of samples, and __getitem__() method to return a data sample at a specific index.



In [ ]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = TorchDataset(train_encodings, y_train)
val_dataset = TorchDataset(val_encodings, y_test)

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir=root+'./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=root+'./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

We then pass our training arguments, dataset, and compute_metrics callback to our Trainer:

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train the model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3386
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1272


Step,Training Loss,Validation Loss


RuntimeError: ignored

In [ ]:
# evaluate the current model after training
trainer.evaluate()

Now that we trained our model, let's save it for inference later:



In [ ]:
# saving the fine tuned model & tokenizer
model_path = root+"transformer2"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Evaluation (on the test dataset)

Now we have a trained model on our dataset, let's try to have some fun with it!

The below function takes a text as a string, tokenizes it with our tokenizer, calculates the output probabilities using softmax function, and returns the actual label:

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    # return probs.argmax() is a tensor. We have to return its item
    return probs.argmax().item()



In [ ]:
y_pred=[]
for text in X_test:
    y_pred.append(get_prediction(text))


In [ ]:
print(y_pred[:10])


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_pred))
